In [ ]:
import numpy as np
import scipy
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
import tensorflow_probability as tfp
tfd = tfp.distributions

In [ ]:
import altair as alt
alt.renderers.enable("notebook")
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

In [ ]:
alpha = 2.
beta = 5.
gamma = tfd.Gamma(concentration=alpha, rate=beta)

def grad_log_like(x):
    with tf.GradientTape() as g:
        tf_x = tf.constant(x, dtype=tf.float32)
        g.watch(tf_x)
        return g.gradient(gamma.log_prob(tf_x), tf_x).numpy()

grad_log_like(np.array([0.3,0.2]))

In [ ]:
def gamma_factory(params):
    return tfp.distributions.Gamma(concentration=params[0], rate=params[1])

def lognormal_factory(params):
    return tfp.distributions.LogNormal(loc=params[0], scale=params[1])

In [ ]:
def tf_grad(params, true_distribution, q_factory, particle_count):
    with tf.GradientTape() as g:
        tf_params = tf.constant(params, dtype=tf.float32)
        g.watch(tf_params)
        q_distribution = q_factory(tf_params)
        tf_x = q_distribution.sample(particle_count)
        y = (1/particle_count) * tf.math.reduce_sum(
            tf.math.log(
                true_distribution.prob(tf_x) / q_distribution.prob(tf_x)
            )
        )
        return g.gradient(y, tf_params).numpy()

In [ ]:
def tf_grad_alt(params, grad_log_p, q_factory, particle_count):
    """
    Here you can supply your own grad log p.
    """
    with tf.GradientTape(persistent=True) as g:
        tf_params = tf.constant(params, dtype=tf.float32)
        g.watch(tf_params)
        q_distribution = q_factory(tf_params)
        tf_x = q_distribution.sample(particle_count)
        q_term = tf.math.reduce_sum(tf.math.log(q_distribution.prob(tf_x)))
    x = tf_x.numpy()
    grad_log_sum_q = g.gradient(q_term, tf_params).numpy()
    grad_x = g.jacobian(tf_x, tf_params).numpy()
    result = np.matmul(grad_log_p(x), grad_x) - grad_log_sum_q
    del g  # Will happen anyway but being explicit to remember.
    return result / particle_count

In [ ]:
tf_grad_alt(np.array([3., 2.]), grad_log_like, gamma_factory, 60)

In [ ]:
def gradient_ascent(params, q_factory, step_size, particle_count, step_count):
    params = np.copy(params)
    history = []
    for _ in range(step_count):
        grad = tf_grad_alt(params, grad_log_like, q_factory, particle_count)
        #grad = tf_grad(params, gamma, q_factory, particle_count)
        params += step_size * grad
        history.append(np.concatenate([np.copy(params), np.copy(grad)]))
    return pd.DataFrame(history)

In [ ]:
history = gradient_ascent(np.array([2., 3.]), gamma_factory, 0.09, 100, 400)
history.plot.line()

In [ ]:
x_grid = np.linspace(0.01, 2)
df = pd.DataFrame({
    "x": x_grid, 
    "gamma": gamma.prob(x_grid).numpy(), 
    "fit": gamma_factory(history.iloc[-1,]).prob(x_grid).numpy()})
df.plot(x="x", y=["gamma", "fit"], kind="line")